# Aprendizem Profunda - DNN
### Tarefa III
1. Rúben Gonçalo Araújo da Silva pg57900   
2. José Luis Fraga Costa pg55970
3. Pedro Miguel Costa Azevedo pg57897
4. Rui Pedro Fernandes Madeira Pinto pg56010

# Implementação

### imports

1. pandas
2. tensorflow
3. sklearn

In [2]:
# Standard Library
import os
import csv

# Data Handling
import pandas as pd
import numpy as np


# Scikit-learn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, classification_report

# TensorFlow / Keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Embedding, Flatten, Dense, Dropout, SimpleRNN, GRU, LSTM, 
    Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D, 
    BatchNormalization, Conv1D, MaxPooling1D
)
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# Hugging Face Transformers
from transformers import (
    AutoTokenizer, 
    TFAutoModelForSequenceClassification, 
    pipeline
)


2025-03-28 02:42:41.644589: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-28 02:42:41.781568: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-28 02:42:43.167117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/rui/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_

### Input dos dados

In [3]:
df = pd.read_csv("data/fixed_dataset_final.csv")

df = df.dropna(subset=["Label"])
df["Text"] = df["Text"].fillna("")
all_ids = df["ID"].fillna("")
test_ids = df["ID"].fillna("")

# Checar balanceamento das classes
print(df['Label'].value_counts())

# Verificar duplicados
print("Duplicados:", df.duplicated(subset='Text').sum())

# Exibir amostra dos dados
print(df.head())



Label
Human    2100
AI       1953
Name: count, dtype: int64
Duplicados: 2
     ID                                               Text  Label
0  D1-1  Advanced electromagnetic potentials are indige...  Human
1  D1-2  This research paper investigates the question ...     AI
2  D1-3  We give an algorithm for finding network encod...  Human
3  D1-4  The paper presents an efficient centralized bi...     AI
4  D1-5  We introduce an exponential random graph model...  Human


### Função pra gravar em csv

In [4]:
def to_csv(results_df, name):
    os.makedirs("score", exist_ok=True)
    path = f"score/{name}.csv"
    filtered_df = results_df[results_df["ID"].notna() & (results_df["ID"] != "")].copy()
    filtered_df['prefix'] = filtered_df['ID'].str.extract(r'(D[0-9])')
    filtered_df['number'] = filtered_df['ID'].str.extract(r'-(\d+)').astype(int)
    filtered_df = filtered_df.sort_values(by=['prefix', 'number'])

    with open(path, 'w') as f:
        f.write("ID\tLabel\n") 
        for index, row in filtered_df.iterrows():
            f.write(f"{row['ID']}\t{row['Label']}\n") 

In [5]:
def predict_and_save(model, X_data, ids, model_name):
    # Generate predictions
    y_pred = (model.predict(X_data) > 0.5).astype(int).flatten()

    # Convert numeric predictions to labels
    labels = ["Human" if label == 0 else "AI" for label in y_pred]
    # Create results DataFrame
    results_df = pd.DataFrame({
        "ID": ids,
        "Label": labels
    })
    to_csv(results_df, model_name)

### Train Test Split (divisão de dados)

In [6]:
# Remove duplicates by keeping the first occurrence
df_unique = df.drop_duplicates(subset=["Text"], keep="first")

# Check how many rows remain
print(f"Original dataset size: {len(df)}, Unique texts: {len(df_unique)}")

# Split the deduplicated dataset
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df_unique["Text"], df_unique["Label"], test_size=0.2, random_state=42, stratify=df_unique["Label"]
)
test_indices = test_texts.index

train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Verify no overlap
print(f"Overlap between train and test after deduplication: {len(set(train_texts) & set(test_texts))}")


Original dataset size: 4053, Unique texts: 4051
Overlap between train and test after deduplication: 0


### Tokenizer

In [7]:
# Tokenize the data
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(train_texts)

X_train_seq = tokenizer.texts_to_sequences(train_texts)
X_val_seq = tokenizer.texts_to_sequences(val_texts)
X_test_seq = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train_seq, maxlen=200)
X_val = pad_sequences(X_val_seq, maxlen=200)
X_test = pad_sequences(X_test_seq, maxlen=200)

# Convert string labels to numeric values
label_map = {"Human": 0, "AI": 1}  # Define mapping
y_train = np.array([label_map[label] for label in train_labels], dtype=np.float32)
y_val = np.array([label_map[label] for label in val_labels], dtype=np.float32)
y_test = np.array([label_map[label] for label in test_labels], dtype=np.float32)

## Dataset de Avaliação

In [8]:
data_set_professor = pd.read_csv("data/dataset1_inputs.csv", sep='\t', on_bad_lines='skip')
data_set_professor["Text"] = data_set_professor["Text"].fillna("")
X_professor_seq = tokenizer.texts_to_sequences(data_set_professor["Text"])
X_professor = pad_sequences(X_professor_seq, maxlen=100)  

### Early Stopping

In [9]:
early_stopping = EarlyStopping(
    monitor='val_loss',     
    patience=5,            
    restore_best_weights=True,           
)

### Reduce_lr

In [10]:
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5, 
    patience=3,
    min_lr=0.00005
)

## CNN

In [11]:
# Modelo CNN mais robusto contra overfitting
model = Sequential([
    Embedding(input_dim=10000, output_dim=64, input_length=200),

    Conv1D(filters=64, kernel_size=5, activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    MaxPooling1D(pool_size=2),
    Dropout(0.4),

    Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.005)),
    BatchNormalization(),
    GlobalMaxPooling1D(),
    Dropout(0.4),

    Dense(32, activation='relu', kernel_regularizer=l2(0.005)),
    Dropout(0.4),

    Dense(1, activation='sigmoid')
])

# Compile com uma taxa de aprendizagem inicial moderada
model.compile(optimizer=Adam(learning_rate=0.0008), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Treine usando EarlyStopping e ReduceLROnPlateau já definidos por você
history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size=64,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping, reduce_lr]
)

Epoch 1/30


/home/rui/miniconda3/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


41/41 ━━━━━━━━━━━━━━━━━━━━ 7s 69ms/step - accuracy: 0.5162 - loss: 2.6903 - val_accuracy: 0.5386 - val_loss: 1.3721 - learning_rate: 8.0000e-04
Epoch 2/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 3s 64ms/step - accuracy: 0.5616 - loss: 1.7720 - val_accuracy: 0.5185 - val_loss: 1.3313 - learning_rate: 8.0000e-04
Epoch 3/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.7174 - loss: 1.2718 - val_accuracy: 0.8287 - val_loss: 1.2484 - learning_rate: 8.0000e-04
Epoch 4/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 54ms/step - accuracy: 0.8484 - loss: 1.0056 - val_accuracy: 0.9275 - val_loss: 1.1635 - learning_rate: 8.0000e-04
Epoch 5/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.9101 - loss: 0.8463 - val_accuracy: 0.9306 - val_loss: 1.0819 - learning_rate: 8.0000e-04
Epoch 6/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - accuracy: 0.9384 - loss: 0.7702 - val_accuracy: 0.9367 - val_loss: 1.0121 - learning_rate: 8.0000e-04
Epoch 7/30
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.9480 - loss: 0.7

## Simple RNN

# Correr Modelos

**Test Set**

In [12]:
test_ids = df.loc[test_indices, "ID"]  # Get IDs for test split only
predict_and_save(model, X_test, test_ids, "cnn")


26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


**Avaliação**

In [13]:
predict_and_save(model, X_professor, data_set_professor["ID"], "cnn_dataset1")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step


## Classification Reports

In [14]:
print(classification_report(y_test, model_cnn.predict(X_test) > 0.5))

NameError: name 'model_cnn' is not defined

In [15]:
# 1. Carregar os arquivos
preds_df = pd.read_csv("score/cnn_dataset1.csv", sep='\t', on_bad_lines='skip')
labels_df = pd.read_csv("data/dataset1_outputs.csv", sep='\t', on_bad_lines='skip')

# 2. Garantir que as colunas corretas estão sendo usadas
# Assumindo que a coluna de previsões se chama 'Pred' e a de verdade se chama 'Label'
y_pred = preds_df['Label']
y_true = labels_df['Label']

# 3. Comparar previsões com os rótulos reais
print("Acurácia:", accuracy_score(y_true, y_pred))
print("Relatório de Classificação:")
print(classification_report(y_true, y_pred))

Acurácia: 0.5
Relatório de Classificação:
              precision    recall  f1-score   support

          AI       0.00      0.00      0.00        15
       Human       0.50      1.00      0.67        15

    accuracy                           0.50        30
   macro avg       0.25      0.50      0.33        30
weighted avg       0.25      0.50      0.33        30



/home/rui/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rui/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/rui/miniconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
